<a href="https://colab.research.google.com/github/Dckstr6/Deep-Learning-Assignments/blob/main/FoDS_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
from random import sample

In [3]:
dataset = pd.read_csv('/content/insurance.csv')

In [4]:
dataset['ones'] = 1
cols = ['ones','age','bmi','children','charges']
dataset = dataset[cols]
dataset.head()

,ones,age,bmi,children,charges
0,1,19,27.900,0,16884.92400
1,1,18,33.770,1,1725.55230
2,1,28,33.000,3,4449.46200
3,1,33,22.705,0,21984.47061
4,1,32,28.880,0,3866.85520


In [5]:
W_N = np.random.randint(1,10,size=(4,))

In [6]:
X = np.array(dataset[['ones','age','bmi','children']])
print(X.shape)

(1338, 4)


In [7]:
X[:,1] = (X[:,1]-np.mean(X[:,1]))/np.std(X[:,1])
X[:,2] = (X[:,2]-np.mean(X[:,2]))/np.std(X[:,2])
X[:,3] = (X[:,3]-np.mean(X[:,3]))/np.std(X[:,3])
print(X.shape)
print(X[0:5,:])

(1338, 4)
[[ 1.         -1.43876426 -0.45332    -0.90861367]
 [ 1.         -1.50996545  0.5096211  -0.07876719]
 [ 1.         -0.79795355  0.38330685  1.58092576]
 [ 1.         -0.4419476  -1.30553108 -0.90861367]
 [ 1.         -0.51314879 -0.29255641 -0.90861367]]


In [8]:
Y = np.array(dataset['charges'])
Y = (Y - np.mean(Y))/np.std(Y)
print(Y.shape)
print(Y[0:5])

(1338,)
[ 0.2985838  -0.95368917 -0.72867467  0.71984298 -0.77680218]


In [ ]:
l = X.shape[0]
f = round(0.7*l)
indices = sample(range(l),f)


X_train = X[indices]
y_train = Y[indices]

X_test = np.delete(X,indices,axis=0)
y_test = np.delete(Y,indices)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(937, 4)
(937,)
(401, 4)
(401,)


## **Normal Solution Attempt**

Equation: W = (X.T * X)^-1 * X.T * Y

In [ ]:
print(W_N)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

[4 9 7 2]


In [ ]:
temp = np.dot((X_train.T),X_train)
temp = np.linalg.inv(temp)
temp = np.dot(temp,(X_train.T))
temp = np.dot(temp,y_train)
print(temp)
W_N = temp
print(W_N)
print(temp.shape)
print(W_N.shape)

[0.00439566 0.29528914 0.1475077  0.06637322]
[0.00439566 0.29528914 0.1475077  0.06637322]
(4,)
(4,)


In [ ]:
def sse_error(W,X,y):
    predictions = np.dot(X,W)
    #print(predictions.shape)
    errors = np.subtract(predictions,y)
    sse = np.sum(errors**2)
    #print(sse)
    return sse

def rmse_error(W,X,y):
    predictions = np.dot(X,W)
    print(predictions.shape)
    errors = np.subtract(predictions,y)
    sse = np.sum(errors**2)
    rmse = np.sqrt(sse/predictions.shape[0])
    return rmse

In [ ]:
print("SSE Errors: ")
print(sse_error(W_N,X_train,y_train))
print(sse_error(W_N,X_test,y_test))
print("RMSE Errors: ")
print(rmse_error(W_N,X_train,y_train))
print(rmse_error(W_N,X_test,y_test))

SSE Errors: 
852.4942784616403
325.861954034106
RMSE Errors: 
(937,)
0.9538409006081894
(401,)
0.9014562256528844


In [ ]:
print(temp)
print(W_N)

[0.00439566 0.29528914 0.1475077  0.06637322]
[0.00439566 0.29528914 0.1475077  0.06637322]


## **GRADIENT** **DESCENT**

In [ ]:
W_GD = np.random.randint(1,10,size=(4,))
print(W_GD)

[1 4 5 5]


GRADIENT DESCENT UPDATE EQUATIONS:

dJ/dWj = -sigma(i)(yi - zi)xij

which means

dJ/dWj = -sigma(i)(yi - np.dot(X,W))xij

(Last product is not matrix multiplication, it is element-wise mul)

In [ ]:
def prediction(W,X):
    pred = np.dot(X,W)
    return pred

In [ ]:
def calc_gradient(W,X,y):
    grad = np.zeros((4,))
    diff = (y - prediction(W,X))
    grad[0] = -1*(np.sum(diff * X[:,0].reshape(X.shape[0],)))
    grad[1] = -1*(np.sum(diff * X[:,1].reshape(X.shape[0],)))
    grad[2] = -1*(np.sum(diff * X[:,2].reshape(X.shape[0],)))
    grad[3] = -1*(np.sum(diff * X[:,3].reshape(X.shape[0],)))
    #print("Gradient: ")
    #print(grad)
    return grad

In [ ]:
def make_update(W,gradient,learning_rate):
    updated_W = np.array(W)
    updated_W = np.subtract(W, learning_rate*gradient)
    return updated_W

In [ ]:
def gradient_descent(W,X,y,iterations,learning_rate):
    for i in range(iterations):
        if(i%50)==0:
          print("==============================================================")
          print("Iteration : %d" % i)
          print("Current Training Error: %f" % (sse_error(W,X,y)))
          gradients = calc_gradient(W,X,y)
          W = make_update(W,gradient=gradients,learning_rate=learning_rate)
          print("Updated W: ")
          print(W)
    return W

In [ ]:
W_GD = gradient_descent(W_GD,X_train,y_train,10,0.001)

Iteration : 0
Current Training Error: 64034.195891
Updated W: 
[0.05802284 0.0565024  0.14343376 0.12166253]
Iteration : 1
Current Training Error: 910.269607
Updated W: 
[0.00451749 0.27621795 0.16227851 0.07890462]
Iteration : 2
Current Training Error: 853.129475
Updated W: 
[0.00390533 0.29233646 0.14966472 0.06762043]
Iteration : 3
Current Training Error: 852.507196
Updated W: 
[0.00428664 0.29486099 0.14784852 0.06653128]
Iteration : 4
Current Training Error: 852.494564
Updated W: 
[0.00437683 0.29522546 0.14755981 0.06639461]
Iteration : 5
Current Training Error: 852.494285
Updated W: 
[0.00439266 0.29527962 0.14751561 0.06637627]
Iteration : 6
Current Training Error: 852.494279
Updated W: 
[0.00439519 0.29528772 0.1475089  0.06637367]
Iteration : 7
Current Training Error: 852.494278
Updated W: 
[0.00439559 0.29528893 0.14750788 0.06637328]
Iteration : 8
Current Training Error: 852.494278
Updated W: 
[0.00439565 0.29528911 0.14750773 0.06637323]
Iteration : 9
Current Training Erro

In [ ]:
print(W_GD)

[0.00439566 0.29528914 0.14750771 0.06637322]


## **Stochastic Gradient Descent**

In [ ]:
W_SGD =  np.random.randint(1,10,size=(4,))

def calc_stoc_grad(W, X, y):
  stocgrad=np.zeros((4,))
  i = random.randint(0,y.shape[0]-1)
  #print(i)
  pred = np.dot(X[i,:] , W)
  #print(pred)
  sdiff = y[i,] - pred
  #print(sdiff)

  stocgrad[0] = -1*(sdiff * X[i,0])
  stocgrad[1] = -1*(sdiff * X[i,1])
  stocgrad[2] = -1*(sdiff * X[i,2])
  stocgrad[3] = -1*(sdiff * X[i,3])

  return stocgrad
#calc_stoc_grad(W_SGD,X_train, y_train)


In [ ]:
def update( W, sgradient , alpha):
  new_W = np.array(W)
  new_W = np.subtract(W, alpha*sgradient)
  return new_W

In [ ]:
def stoc_gradient_descent(W,X,y,iterations,n):
  for i in range(iterations):
    if (i%50)==0:
      print("########################################################")
      print("ITERATION NO: %d" % i )
      print("CURRENT TRAINING ERROR (SSE) : %f" % (sse_error(W,X,y)))
      print("CURRENT TRAINING ERROR (RMSE) : %f" % (rmse_error(W,X,y)))
      gradients=calc_stoc_grad(W,X,y)
      W = update(W, sgradient = gradients, alpha = n)
      print("NEW W: ", W)
  return W

In [ ]:
W_SGD = stoc_gradient_descent(W_SGD, X_train, y_train, 10000, 0.001)

Streaming output truncated to the last 5000 lines.
NEW W:  [-0.03251076  0.30453415  0.12900223  0.0497982 ]
########################################################
ITERATION NO: 9167
CURRENT TRAINING ERROR (SSE) : 854.430163
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.954923
NEW W:  [-0.03305424  0.3052774   0.12876449  0.05029202]
########################################################
ITERATION NO: 9168
CURRENT TRAINING ERROR (SSE) : 854.473069
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.954947
NEW W:  [-0.03343369  0.30520194  0.12911794  0.05000703]
########################################################
ITERATION NO: 9169
CURRENT TRAINING ERROR (SSE) : 854.495135
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.954960
NEW W:  [-0.03132129  0.30471959  0.13069     0.04984064]
########################################################
ITERATION NO: 9170
CURRENT TRAINING ERROR (SSE) : 854.293372
(937,)
CURRENT TRAINING ERROR (RMSE) : 0.954847
NEW W:  [-0.03178949  0.30439313  0.13068105  0.04987

In [ ]:
print(W_SGD)

[1.01329001e-04 2.76952181e-01 1.57423462e-01 6.93156444e-02]
